In [2]:
%pip install openai -U --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
from openai import AzureOpenAI
import time
import json
import os
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
    api_version=os.getenv("OPENAI_API_VERSION_ASSISTANTS"),
    api_key=os.getenv("AZURE_OPENAI_KEY_ASSISTANTS"),  
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT_ASSISTANTS")
)

In [3]:
# Create an assistant
assistant = client.beta.assistants.create(
    name="Math Assist",
    instructions="You are an AI assistant that can write code to help answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4"
)

# Create a thread
thread = client.beta.threads.create()

# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status

# Wait till the assistant has responded
while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    status = run.status

messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

NotFoundError: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}